<center>
    <h2>Detection des anomalies</h2>
    <hr/>
</center>

### Chargement des bibliothèques

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
import seaborn as sns
from jupyterthemes import jtplot
import datetime
import networkx as nx

jtplot.style(theme = 'onedork')

### Chargement des données

In [2]:
Assure = pd.read_csv('../Data/assure.csv', sep = ';')
BonusMalus = pd.read_csv('../Data/classBonusMalus.csv', sep = ';')
Epave = pd.read_csv('../Data/epave.csv', sep = ";")
Marque = pd.read_excel('../Data/marque.xlsx', sep = ';')
Police = pd.read_csv('../Data/Police.csv', sep = ';')
Sinistre = pd.read_csv('../Data/Sinistre.csv', sep = ';')
Usage = pd.read_excel('../Data/Usage.xlsx', sep = ';')
UsageCGA = pd.read_csv('../Data/UsageCGA.csv', sep = ';')
Vehicule = pd.read_csv('../Data/vehicule.csv', sep = ';')

C:\Users\ASUS\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\ASUS\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Anomalie 1 : Vehicules déclarées dans plusieurs polices

In [38]:
poop = BonusMalus.groupby('vehicule_id')[['CONTRAT_EN_COURS']].sum()

#plt.figure(figsize = (20, 10))
sns.countplot(data = poop[poop['CONTRAT_EN_COURS'] > 1], x = 'CONTRAT_EN_COURS')
plt.xlabel('\nNumber of Operative Policies')
plt.ylabel('Number of Vehicles\n')
plt.xticks()
plt.yticks()
plt.title('Abnormal Behaviour Found in a Number of Registered Vehicles\n', 
          fontdict = {'fontsize': 20});

### Anomalie 2 : Assurés avec un nombre suspect de véhicules

In [54]:
poop = BonusMalus.groupby('assure_id')[['vehicule_id']].count()
poop = poop.sort_values(by = 'vehicule_id', ascending = False).reset_index()
tmp = poop['assure_id'].head(10).values

sns.barplot(data = poop.head(10), x = 'assure_id', y = 'vehicule_id', order = list(tmp))
plt.ylabel('Number of Registered Vehicles\n')
plt.xlabel('\nInsured ID')
plt.title('Abnormal Behaviour Detected in a Number of Clients\n', 
          fontdict = {'fontsize': 20});

Le client *53968* a un nombre suspect de véhicules.<br/>
NB : Même pour un contrat flotte 400 véhicule est un nombre énorme.

In [83]:
poop = BonusMalus.merge(Police, left_on = 'police_id', right_on = 'id', how = 'left')
poop = poop.where(poop['assure_id'] == 53968).dropna(how = 'all')

sns.countplot(data = poop, x = 'CONTRAT_EN_COURS', hue = 'typePolice')
plt.ylabel('Number of Policies')
plt.xlabel('')
plt.legend(labels = ['Individual', 'Fleet'], title = 'Type of Policy')
plt.xticks([0, 1], ['Inoperative Policies', 'Operative Policies'])
plt.title('\nAbnormal Behaviour Detected in a Number of Clients\n', 
          fontdict = {'fontsize': 25});

La plupart (presque tous) des polices sont de type *Individuel*

In [73]:
poop = BonusMalus.where(BonusMalus['assure_id'] == 53968).dropna(how = 'all')
poop = poop = poop.where(poop['CONTRAT_EN_COURS'] == 1).dropna(how = 'all')
polices = poop['police_id'].values
poop = Police.where(Police['id'].isin(polices)).dropna(how = 'all')[['codeCompagnie', 'codeAgence', 'naturePolice', 'Etat_Police']]

poop.head()

,codeCompagnie,codeAgence,naturePolice,Etat_Police
989,15.0,9.0,R,R
2081,15.0,9.0,R,R
2458,15.0,9.0,R,V
2468,15.0,9.0,R,R
2469,15.0,9.0,R,R


In [85]:
poop = BonusMalus.merge(Police, left_on = 'police_id', right_on = 'id', how = 'left')

sns.countplot(data = poop, x = 'CONTRAT_EN_COURS', hue = 'Etat_Police')
plt.ylabel('Number of Policies\n')
plt.xlabel('\nCONTRAT_EN_COURS')
plt.title('\nInconsistencies in Certain Variables\n', 
          fontdict = {'fontsize': 25});

On n'a pris que les contrats qui sont en cours et on a trouvé des contrats résiliés. <br/>
Il n'y a pas de cohérence entre la varibale `CONTRAT_EN_COURS` et `Etat_Police`

### Anomalie 3 : Simulation des accidents (graphes)

In [14]:
poop = Sinistre[['cga_police_id', 'numeroDePoliceCompagnieAdverse']]
poop = poop.merge(Police[['id', 'numPolice']], 
                  left_on = 'numeroDePoliceCompagnieAdverse', 
                  right_on = 'numPolice', 
                  how = 'left')

poop.dropna(inplace = True)
poop.drop(columns = ['numeroDePoliceCompagnieAdverse', 'numPolice'], inplace = True)
poop.rename(columns = {
    'cga_police_id': 'policeId (A)',
    'id': 'policeId (B)'
}, inplace = True)

poop.head()

,policeId (A),policeId (B)
2,174573,101562.0
7,174619,2066580.0
27,174803,205494.0
29,174811,1242195.0
37,174852,1359974.0


On a créé une dataframe décrivant les deux partis d'un sinistre.

In [16]:
nodes = np.array(poop.values).reshape((poop.shape[0]*poop.shape[1], ))
edges = [(poop.iloc[i, 0], poop.iloc[i, 1]) for i in range(poop.shape[0])]

Les noeuds de notre graphe représentent les polices et les arêtes représentent les sinistres. Les poids des arêtes sont les nombres de sinistres survenus.

In [17]:
weighted_edges = []

for e in edges:
    weight = 0
    tmp = edges.copy()
    tmp.remove(e)
    for f in tmp:
        if ((e[0] == f[0]) & (e[1] == f[1])) | ((e[1] == f[0]) & (e[0] == f[1])):
            weight += 1
    weighted_edges.append((e[0], e[1], weight))

Création du graphe

In [22]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_weighted_edges_from(weighted_edges)

Vu le nombre énorme de noeuds qu'on aura, une représentation graphique de ce graphe sera très compliqué à comprendre. Notre but dans cette partie est de détetecter s'il existe un circuit (Un circuit peut dévoiler un groupe d'assuré qui sont entrain de simuler des acidents entre eux). Ainsi, on affichera seulement les circuits.

In [23]:
flatten = lambda l: [item for sublist in l for item in sublist]

edges = []
for cycle in nx.cycle_basis(G):
    if len(cycle) != 1:
        for i in range(len(cycle) - 1):
            edges.append((cycle[i], cycle[i + 1]))
        edges.append((cycle[0], cycle[-1]))
        
nodes = flatten([[e[0], e[1]] for e in edges])

In [24]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

pos = nx.spring_layout(G)

if nodes == []:
    print('No cycles')
else:
    plt.figure(figsize = (20, 10))
    nx.draw_networkx(G, pos = pos, with_labels = True, node_color = '#7f9a49', node_size = 500, 
                     alpha = 1, width = 1, edge_color = 'black')

No cycles


Exemple d'extraction de circuits :

In [22]:
G = nx.Graph()
nodes = [k for k in range(1, 10)]
edges = [(1, 2), (2, 3), (3, 1), (4, 5), (5, 6), (6, 7), (7, 4), (1, 7), (8, 9), (8, 8)]
G.add_nodes_from(nodes)
G.add_edges_from(edges)

pos = nx.spring_layout(G)
%matplotlib qt
plt.figure(figsize = (15, 7))
nx.draw_networkx(G, pos = pos, with_labels = True, node_color = '#7f9a49', node_size = 800, 
                 alpha = 1, width = 1, edge_color = 'black')
plt.grid(False)
plt.title('Fraude Detection Using Graph Theory\n')

Text(0.5, 1.0, 'Fraude Detection Using Graph Theory\n')

In [23]:
flatten = lambda l: [item for sublist in l for item in sublist]

edges = []
for cycle in nx.cycle_basis(G):
    if len(cycle) != 1:
        for i in range(len(cycle) - 1):
            edges.append((cycle[i], cycle[i + 1]))
        edges.append((cycle[0], cycle[-1]))
        
nodes = flatten([[e[0], e[1]] for e in edges])

In [24]:
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

pos = nx.spring_layout(G)

plt.figure(figsize = (15, 7))
nx.draw_networkx(G, pos = pos, with_labels = True, node_color = '#7f9a49', node_size = 800, 
                 alpha = 1, width = 1, edge_color = 'black')
plt.grid(False)
plt.title('Fraude Detection Using Graph Theory\n')

Text(0.5, 1.0, 'Fraude Detection Using Graph Theory\n')